# *Desarrollo notebook 3* 

### Codificación de las variables categoricas, escalado y modelo

In [6]:
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn import metrics


### **Lectura de datos del preprocesado missing outlier**

In [7]:
df_train = pd.read_csv('/Users/miguelflores/Desktop/CSV/train_df_preprocessing_missing_outlier.csv').set_index("SK_ID_CURR")
df_test = pd.read_csv('/Users/miguelflores/Desktop/CSV/test_df_preprocessing_missing_outlier.csv').set_index("SK_ID_CURR")

In [8]:
df_train.columns

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE',
       ...
       'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR', 'NWEEKDAY_PROCESS_START', 'TARGET'],
      dtype='object', length=121)

In [10]:
df_train.dtypes

NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                    int64
FLAG_OWN_REALTY                 int64
CNT_CHILDREN                    int64
                               ...   
AMT_REQ_CREDIT_BUREAU_MON     float64
AMT_REQ_CREDIT_BUREAU_QRT     float64
AMT_REQ_CREDIT_BUREAU_YEAR    float64
NWEEKDAY_PROCESS_START          int64
TARGET                          int64
Length: 121, dtype: object

## **Codificación de la variable objetivo**

#####   1 - Cliente con dificultades de pago: Pago atrasado de más de X días en al menos una de las primeras Y cuotas del préstamo
#####   0 - Cliente sin dificultades de pago

In [14]:
count_0 = df_train['TARGET'].value_counts()[0]
count_1 = df_train['TARGET'].value_counts()[1]

print(f"0   {count_0}")
print(f"1   {count_1}")

0   226148
1   19860


In [15]:
X_train = df_train.drop('TARGET',axis=1)
X_test = df_test.drop('TARGET',axis=1)
y_train = df_train['TARGET']
y_test = df_test['TARGET']

## **Codificación del resto de variables categoricas**

#### Por ejemplo, elijo codificarlas con one-hot-encoding todas. Se podría elegir codificar algunas de una forma y otras de otra. Por ejemplo, variables con pocas categorías con one hot encoding  variables con más categorias con TargetEncoding


#### En este caso codificamos las variables categóricas unicamente, debido a que sus variables no contienen más de 25 valores por lo que es óptimo realizar este cambio

In [16]:
list_columns_cat = list(X_train.select_dtypes("object", "category").columns)
list_other = list(set(X_train.columns)-set(list_columns_cat))

In [17]:
ohe = ce.OneHotEncoder(cols=list_columns_cat)
model = ohe.fit(X_train, y_train)

In [18]:
model

OneHotEncoder(cols=['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'NAME_TYPE_SUITE',
                    'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
                    'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
                    'OCCUPATION_TYPE', 'ORGANIZATION_TYPE',
                    'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE',
                    'WALLSMATERIAL_MODE'])

In [19]:
X_train_t = model.transform(X_train, y_train)
X_test_t = model.transform(X_test, y_test)

In [20]:


len(list(X_train_t.columns))



240

In [21]:
X_train_t.dtypes.to_dict()

{'NAME_CONTRACT_TYPE_1': dtype('int64'),
 'NAME_CONTRACT_TYPE_2': dtype('int64'),
 'CODE_GENDER_1': dtype('int64'),
 'CODE_GENDER_2': dtype('int64'),
 'CODE_GENDER_3': dtype('int64'),
 'FLAG_OWN_CAR': dtype('int64'),
 'FLAG_OWN_REALTY': dtype('int64'),
 'CNT_CHILDREN': dtype('int64'),
 'AMT_INCOME_TOTAL': dtype('float64'),
 'AMT_CREDIT': dtype('float64'),
 'AMT_ANNUITY': dtype('float64'),
 'AMT_GOODS_PRICE': dtype('float64'),
 'NAME_TYPE_SUITE_1': dtype('int64'),
 'NAME_TYPE_SUITE_2': dtype('int64'),
 'NAME_TYPE_SUITE_3': dtype('int64'),
 'NAME_TYPE_SUITE_4': dtype('int64'),
 'NAME_TYPE_SUITE_5': dtype('int64'),
 'NAME_TYPE_SUITE_6': dtype('int64'),
 'NAME_TYPE_SUITE_7': dtype('int64'),
 'NAME_TYPE_SUITE_8': dtype('int64'),
 'NAME_INCOME_TYPE_1': dtype('int64'),
 'NAME_INCOME_TYPE_2': dtype('int64'),
 'NAME_INCOME_TYPE_3': dtype('int64'),
 'NAME_INCOME_TYPE_4': dtype('int64'),
 'NAME_INCOME_TYPE_5': dtype('int64'),
 'NAME_INCOME_TYPE_6': dtype('int64'),
 'NAME_INCOME_TYPE_7': dtype('in

## **Escalado de las variables**

#### En este caso se realizo el escalado de las variables, con el fin de visualizar dichas métricas, ya que en algunos algoritmos no es necesario estandarizar variables, aunque no pasa nada si se hace.

In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
model_scaled = scaler.fit(X_train_t)
X_train_scaled = pd.DataFrame(scaler.transform(X_train_t), columns=X_train_t.columns, index=X_train_t.index)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_t), columns=X_test_t.columns, index=X_test.index)

In [24]:
X_train_scaled.describe()

,NAME_CONTRACT_TYPE_1,NAME_CONTRACT_TYPE_2,CODE_GENDER_1,CODE_GENDER_2,CODE_GENDER_3,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NWEEKDAY_PROCESS_START
count,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,...,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05,2.460080e+05
mean,1.730086e-17,1.371938e-18,4.627042e-17,8.513238e-18,1.497398e-18,-2.691526e-17,9.794917e-17,-6.457136e-17,-1.184560e-16,5.154156e-17,...,-5.300014e-18,8.087215e-18,-3.895583e-18,2.087512e-17,8.000567e-18,4.416197e-17,6.444500e-17,6.221379e-17,-1.269404e-17,-3.526604e-17
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,...,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-3.080851e+00,-3.245857e-01,-1.387247e+00,-7.208392e-01,-2.851295e-03,-7.174304e-01,-1.505409e+00,-5.787491e-01,-5.529932e-01,-1.376469e+00,...,-2.470037e-02,-2.245669e-02,-1.792292e-02,-7.048615e-02,-5.789143e-02,-1.548934e-01,-2.691358e-01,-3.961795e-01,-1.006786e+00,-1.410546e+00
25%,3.245857e-01,-3.245857e-01,-1.387247e+00,-7.208392e-01,-2.851295e-03,-7.174304e-01,-1.505409e+00,-5.787491e-01,-2.176134e-01,-8.177746e-01,...,-2.470037e-02,-2.245669e-02,-1.792292e-02,-7.048615e-02,-5.789143e-02,-1.548934e-01,-2.691358e-01,-3.961795e-01,-4.399625e-01,-8.524510e-01
50%,3.245857e-01,-3.245857e-01,7.208522e-01,-7.208392e-01,-2.851295e-03,-7.174304e-01,6.642711e-01,-5.787491e-01,-8.207125e-02,-2.099715e-01,...,-2.470037e-02,-2.245669e-02,-1.792292e-02,-7.048615e-02,-5.789143e-02,-1.548934e-01,-2.691358e-01,-3.961795e-01,-4.399625e-01,-2.943560e-01
75%,3.245857e-01,-3.245857e-01,7.208522e-01,1.387272e+00,-2.851295e-03,1.393863e+00,6.642711e-01,8.111074e-01,1.299305e-01,5.197387e-01,...,-2.470037e-02,-2.245669e-02,-1.792292e-02,-7.048615e-02,-5.789143e-02,-1.548934e-01,-2.691358e-01,-3.961795e-01,6.936836e-01,8.218339e-01
max,3.245857e-01,3.080851e+00,7.208522e-01,1.387272e+00,3.507178e+02,1.393863e+00,6.642711e-01,2.582852e+01,4.511550e+02,8.568284e+00,...,4.048522e+01,4.453016e+01,5.579449e+01,5.176855e+01,8.731484e+01,4.168428e+01,3.118711e+01,3.255112e+01,1.316379e+01,1.938024e+00


## **Conclusiones**

#### A lo largo de este trabajo, se visualizaron los diferentes rendimientos que tenían las variables en este conjunto de datos, consideramos que nos encontramos en una etapa de análisis temprana para cumplir uno de los objetivos y contestar a la pregunta **¿Hay algún tipo de clientes más propenso a no devolver un préstamo?**. Sin embargo, consideramos que por medio de visualizar las gráficas, obtener los valores de Cramérs, visualizar la matriz de correlación, entre otros aspectos, somos capaces de identificar y determinar patrones en los datos, los cuales nos indicaran y nos daran una una idea de la capacidad de solicitantes para cumplir con sus obligaciones financieras. Los pasos siguientes a esta etapa de análisis es generar el modelo y con este ratificar que los usuarios capaces de saldar el préstamo no sean rechazados, a la par de detectar perfiles con dificultades para cubrir la deuda. 

#### De igual manera es importante el siempre tener presente que no se debe de menospreciar ningún tipo de varibale sin importar el significado de esta, ya que como científicos de datos, debemos de ir más alla de la lógica y debemos de saber detectar lo que nos llevara al cumplimiento de nuestros objetivos, que es generar un modelo predictivo con poca instancia al error y una gran generación de resultados. 